# ABUSIVE E-MAIL CLASSIFICTION

In [1]:
#Python Packages
import pandas as pd

In [2]:
#EMAIL DATASET
D = pd.read_csv('C:/Users/prath/Downloads/emails1 (1).txt')

# EDA (EXPLORATORY DATA ANALYSIS)

In [3]:
data = D.copy()

In [4]:
data.shape

(48076, 5)

In [5]:
#FIND ANY DATA WHY IS NULL OR NOT.
data.isnull().sum()

Unnamed: 0    0
filename      0
Message-ID    0
content       0
Class         0
dtype: int64

In [6]:
#CHECK ANY DUPLIACTED DATA IN THE DATASET
data.duplicated().sum()

0

In [7]:
#DROP UNNESSASSRY COLUMNS
data = data.drop('Unnamed: 0',axis=1).drop('Class',axis=1)

In [8]:
#RENAME COLUMNS TO SIMPLER FORM
data = data.rename({'filename':'Name'},axis=1)
data = data.rename({'content':'Content'},axis=1)
data = data.rename({'Message-ID':'Mail-ID'},axis=1)

In [9]:
data.head()

,Name,Mail-ID,Content
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\n\n\n\n\nJohn J Lavorato@excelr\n11/1...
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,Gentlemen:\nThe following champagne is availab...
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\n\n\n\n\nJohn J Lavorato@excelr\n12/23...


In [10]:
import cleantext

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [11]:
for i in range (len(data.Content)):
    x = cleantext.clean(data.Content[i])
    data.Content[i] = x

In [12]:
from sklearn import preprocessing
LE = preprocessing.LabelEncoder()
data['Original'] = LE.fit_transform(D['Class'])

In [13]:
data.head()

,Name,Mail-ID,Content,Original
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\njohn j lavorato@excelr\n11/18/2000 0...,0
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you,0
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,gentlemen:\nthe following champagne is availab...,0
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...,0
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\njohn j lavorato@excelr\n12/23/2000 10...,0


# MODEL BUILDING - 1 (SENTIMENTAL ANALYSIS)

In [14]:
from textblob import TextBlob

In [15]:
data['SentimentalAnalysis'] = pd.Series()

<ipython-input-15-5f82b595971f>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['SentimentalAnalysis'] = pd.Series()


In [16]:
#0 = ABUSIVE and 1 = NON-ABUSIVE
for j in range(len(data.Content)):
    x = TextBlob(data.Content[j]).sentiment
    if x.polarity < 0.0:
        data.SentimentalAnalysis[j] = 0
    else:
        data.SentimentalAnalysis[j] = 1

<ipython-input-16-08d29f58434a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.SentimentalAnalysis[j] = 0
<ipython-input-16-08d29f58434a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.SentimentalAnalysis[j] = 1


In [17]:
data.head()

,Name,Mail-ID,Content,Original,SentimentalAnalysis
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\njohn j lavorato@excelr\n11/18/2000 0...,0,0.0
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you,0,0.0
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,gentlemen:\nthe following champagne is availab...,0,1.0
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...,0,0.0
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\njohn j lavorato@excelr\n12/23/2000 10...,0,0.0


# MODEL BUILDING-2 (BAG-OF-WORDS)

In [18]:
#ABUSIVE WORDS DATASET
ABW = pd.read_csv('C:/Users/prath/Downloads/abusive_words.csv')

In [19]:
ABW.head(5)

,abuses
0,shit
1,fuck
2,damn
3,bitch
4,crap


In [20]:
min_occurance = 1
data['BagOfWords'] = pd.Series()

<ipython-input-20-dc2e0a5f2135>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['BagOfWords'] = pd.Series()


In [21]:
#0 FOR ABUSIVE AND 1 FOR NON-ABUSIVE
occurance = []
for k in range(len(data.BagOfWords)):
    for l in ABW.abuses:
        if l in data.Content[k].split():
            occurance.append(0)
        else:
            occurance.append(1)
    if occurance.count(0) >= min_occurance:
        data.BagOfWords[k] = 0
    else:
        data.BagOfWords[k] = 1
    occurance.clear()

<ipython-input-21-cf226e05eab8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.BagOfWords[k] = 0
<ipython-input-21-cf226e05eab8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.BagOfWords[k] = 1


In [22]:
data.head()

,Name,Mail-ID,Content,Original,SentimentalAnalysis,BagOfWords
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\njohn j lavorato@excelr\n11/18/2000 0...,0,0.0,0.0
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you,0,0.0,0.0
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,gentlemen:\nthe following champagne is availab...,0,1.0,1.0
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...,0,0.0,0.0
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\njohn j lavorato@excelr\n12/23/2000 10...,0,0.0,0.0


# MODEL-3 (VADER SENTIMENTAL ANALYSIS)

In [23]:
!pip install vaderSentiment

In [24]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [25]:
data['VADER'] = pd.Series()

<ipython-input-25-37cd094e9122>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['VADER'] = pd.Series()


In [26]:
analyzer = SentimentIntensityAnalyzer()
for i in range(len(data.Content)):
    vs = analyzer.polarity_scores(data.Content[i])
    if (list(vs.values())[0])>0:
        data.VADER[i] = 0
    else:
        data.VADER[i] = 1

<ipython-input-26-79299fa9acb1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.VADER[i] = 0
<ipython-input-26-79299fa9acb1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.VADER[i] = 1


In [27]:
data.head()

,Name,Mail-ID,Content,Original,SentimentalAnalysis,BagOfWords,VADER
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\njohn j lavorato@excelr\n11/18/2000 0...,0,0.0,0.0,0.0
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you,0,0.0,0.0,0.0
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,gentlemen:\nthe following champagne is availab...,0,1.0,1.0,1.0
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...,0,0.0,0.0,0.0
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\njohn j lavorato@excelr\n12/23/2000 10...,0,0.0,0.0,0.0


# MODEL-4 (SVM)

In [28]:
datatrain = data.head(33600)
datatest = data.tail(14400)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.8, sublinear_tf = True, use_idf = True)
train_vectors = vectorizer.fit_transform(datatrain['Content'])
test_vectors = vectorizer.transform(datatest['Content'])

In [31]:
import time
from sklearn import svm
from sklearn.metrics import classification_report

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, datatrain['Original'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(datatest['Original'], prediction_linear, output_dict=True)

Training time: 486.383442s; Prediction time: 91.211272s


C:\Users\prath\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
data['SVM'] = pd.Series()

<ipython-input-32-7b171abca0d5>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['SVM'] = pd.Series()


In [33]:
for i in range(len(data.Content)):
    review = data.Content[i]
    review_vector = vectorizer.transform([review])
    data.SVM[i] = (classifier_linear.predict(review_vector))

<ipython-input-33-4bff51c28569>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.SVM[i] = (classifier_linear.predict(review_vector))


In [34]:
data.head()

,Name,Mail-ID,Content,Original,SentimentalAnalysis,BagOfWords,VADER,SVM
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\njohn j lavorato@excelr\n11/18/2000 0...,0,0.0,0.0,0.0,0.0
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you,0,0.0,0.0,0.0,0.0
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,gentlemen:\nthe following champagne is availab...,0,1.0,1.0,1.0,0.0
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...,0,0.0,0.0,0.0,0.0
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\njohn j lavorato@excelr\n12/23/2000 10...,0,0.0,0.0,0.0,0.0


# MODEL - 5 (NAIVE BAYES) DEEP LEARNING MODEL

In [35]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
x = data['Content']
y = data['Original']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [37]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(x)
x_test = vec.transform(x_test)

In [38]:
model = MultinomialNB()
model.fit(x, y)

MultinomialNB()

In [39]:
data['NAIVE'] = pd.Series()

<ipython-input-39-049bf9b66335>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['NAIVE'] = pd.Series()


In [40]:
#0 = ABUSIVE and 1 = NON-ABUSIVE
for i in range(len(data.Content)):
    A = model.predict(vec.transform([data.Content[i]]))
    if A[0] == 0:
        data.NAIVE[i] = 0
    else:
        data.NAIVE[i] = 1

<ipython-input-40-c6143e6c526a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.NAIVE[i] = 1
<ipython-input-40-c6143e6c526a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.NAIVE[i] = 0


In [41]:
data.head()

,Name,Mail-ID,Content,Original,SentimentalAnalysis,BagOfWords,VADER,SVM,NAIVE
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\njohn j lavorato@excelr\n11/18/2000 0...,0,0.0,0.0,0.0,0.0,1.0
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you,0,0.0,0.0,0.0,0.0,0.0
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,gentlemen:\nthe following champagne is availab...,0,1.0,1.0,1.0,0.0,0.0
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...,0,0.0,0.0,0.0,0.0,0.0
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\njohn j lavorato@excelr\n12/23/2000 10...,0,0.0,0.0,0.0,0.0,1.0


# MODEL - 6 (RANDOM FOREST)

In [42]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
x = data['Content']
y = data['Original']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [44]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(x)
x_test = vec.transform(x_test)

In [45]:
kfold = KFold(n_splits=10)
model = RandomForestClassifier(n_estimators=100, max_features=3)
model.fit(x,y)

RandomForestClassifier(max_features=3)

In [46]:
data['RF'] = pd.Series()

<ipython-input-46-ad1ea022f8d8>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['RF'] = pd.Series()


In [47]:
#0 = ABUSIVE and 1 = NON-ABUSIVE
for i in range(len(data.Content)):
    A = model.predict(vec.transform([data.Content[i]]))
    if A[0] == 0:
        data.RF[i] = 0
    else:
        data.RF[i] = 1

<ipython-input-47-26f924bfad95>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.RF[i] = 0
<ipython-input-47-26f924bfad95>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.RF[i] = 1


In [48]:
data.head()

,Name,Mail-ID,Content,Original,SentimentalAnalysis,BagOfWords,VADER,SVM,NAIVE,RF
0,arnold-j/_sent_mail/34.,<15267340.1075857594923.JavaMail.evans@thyme>,eat shit\njohn j lavorato@excelr\n11/18/2000 0...,0,0.0,0.0,0.0,0.0,1.0,0.0
1,arnold-j/_sent_mail/517.,<15539583.1075857652152.JavaMail.evans@thyme>,fuck you,0,0.0,0.0,0.0,0.0,0.0,0.0
2,arnold-j/_sent_mail/743.,<4339799.1075857657711.JavaMail.evans@thyme>,gentlemen:\nthe following champagne is availab...,0,1.0,1.0,1.0,0.0,0.0,0.0
3,arnold-j/_sent_mail/766.,<25574783.1075857658215.JavaMail.evans@thyme>,sorry i've taken so long...just been trying to...,0,0.0,0.0,0.0,0.0,0.0,0.0
4,arnold-j/_sent_mail/797.,<19506151.1075857658895.JavaMail.evans@thyme>,asshole\njohn j lavorato@excelr\n12/23/2000 10...,0,0.0,0.0,0.0,0.0,1.0,0.0


# MODEL VALIDATION

In [49]:
from sklearn.metrics import accuracy_score

In [50]:
#SENTIMENTAL ANALYSIS ACCURACY
def SentimentalAccuracy():
    accuracy = accuracy_score(data.Original,data.SentimentalAnalysis)
    return accuracy*100
SentimentalAccuracy()

81.25468008985773

In [51]:
#BAG-OF-WORDS ACCURACY
def BagofwordsAccuracy():
    accuracy = accuracy_score(data.Original,data.BagOfWords)
    return accuracy*100
BagofwordsAccuracy()

91.9336051252184

In [52]:
#VADER SENTIMENTAL ANALYSIS ACCURACY
def VADER():
    accuracy = accuracy_score(data.Original,data.VADER)
    return accuracy*100
VADER()

50.846576254264086

In [53]:
#SVM ACCURACY
def SVM():
    accuracy = accuracy_score(data.Original,data.SVM)
    return accuracy*100
SVM()

99.32814710042433

In [54]:
#NAIVE BAYES ACCURACY
def NB():
    accuracy = accuracy_score(data.Original,data.NAIVE)
    return accuracy*100
NB()

97.20858640485898

In [55]:
#RANDOM FOREST ACCURACY
def RF():
    accuracy = accuracy_score(data.Original,data.RF)
    return accuracy*100
RF()

99.65679341043348